In [1]:
#load files needed
import pandas as pd

In [2]:
allEventsData=pd.read_csv("allEventsData.csv", encoding="utf-8", index_col=0) 
allEventsData['datetest'] = pd.to_datetime(allEventsData['timestamp'])

In [169]:
allEventsData.head()

,id,matchid,outcome,team,teamid,timestamp,type,winner,datetest
0,1543551217,988751,1,NONE,NONE,2018-07-28T15:05:32.59,34,team_1,2018-07-28 15:05:32.590
1,1357993239,988751,1,NONE,NONE,2018-07-28T15:09:55.17,34,team_1,2018-07-28 15:09:55.170
2,1634877849,988751,1,NONE,NONE,2018-07-28T16:00:12.288,32,team_1,2018-07-28 16:00:12.288
3,1097243024,988751,1,NONE,NONE,2018-07-28T16:00:12.288,32,team_1,2018-07-28 16:00:12.288
4,1184172299,988751,1,team_0,t2081,2018-07-28T16:00:13.432,1,team_1,2018-07-28 16:00:13.432


In [170]:
print(allEventsData.dtypes)
print(len(allEventsData))

id                    int64
matchid               int64
outcome               int64
team                 object
teamid               object
timestamp            object
type                  int64
winner               object
datetest     datetime64[ns]
dtype: object
137699


In [205]:
allMatchData = pd.read_csv("allMatchData.csv", encoding="utf-8", index_col=0) 
allMatchData['matchdate'] = pd.to_datetime(allMatchData['matchdata'])

In [172]:
allMatchData.head()

,matchdata,matchid,team_0_s_accurate_back_zone_pass,team_0_s_accurate_chipped_pass,team_0_s_accurate_corners_intobox,team_0_s_accurate_cross,team_0_s_accurate_cross_nocorner,team_0_s_accurate_flick_on,team_0_s_accurate_freekick_cross,team_0_s_accurate_fwd_zone_pass,...,team_1_s_total_throws,team_1_s_total_yel_card,team_1_s_touches,team_1_s_touches_in_opp_box,team_1_s_unsuccessful_touch,team_1_s_won_contest,team_1_s_won_corners,team_1_s_won_tackle,team_1_teamRef,winner
0,20180728T160000+0100,988751,140.0,17.0,1.0,4.0,3.0,0.0,0.0,151.0,...,18.0,1.0,530.0,24.0,15.0,6.0,9.0,15.0,t2089,team_1
1,20180728T160000+0100,988750,225.0,30.0,0.0,1.0,1.0,1.0,0.0,166.0,...,16.0,1.0,431.0,9.0,8.0,5.0,6.0,12.0,t4950,team_0
2,20180729T160000+0100,988753,114.0,6.0,1.0,2.0,1.0,2.0,0.0,60.0,...,27.0,2.0,928.0,24.0,14.0,6.0,5.0,13.0,t212,team_1
3,20180729T160000+0100,988752,207.0,13.0,0.0,5.0,5.0,2.0,0.0,234.0,...,24.0,4.0,445.0,10.0,12.0,4.0,4.0,12.0,t280,team_0
4,20180729T160000+0100,988754,72.0,12.0,5.0,9.0,4.0,3.0,0.0,128.0,...,32.0,2.0,619.0,26.0,13.0,5.0,3.0,10.0,t5666,team_0


What are good indicators that describe the match?

- team_0|1_games_won_before
- stats of game before in same position?

In [174]:
mlMatchData = allMatchData[['matchid','matchdata','team_0_teamRef','team_1_teamRef','winner']].copy()
mlMatchData['matchdate'] = pd.to_datetime(mlMatchData['matchdata'])

In [175]:
mlMatchData.head()

,matchid,matchdata,team_0_teamRef,team_1_teamRef,winner,matchdate
0,988751,20180728T160000+0100,t2081,t2089,team_1,2018-07-28 15:00:00
1,988750,20180728T160000+0100,t199,t4950,team_0,2018-07-28 15:00:00
2,988753,20180729T160000+0100,t2002,t212,team_1,2018-07-29 15:00:00
3,988752,20180729T160000+0100,t857,t280,team_0,2018-07-29 15:00:00
4,988754,20180729T160000+0100,t4948,t5666,team_0,2018-07-29 15:00:00


In [176]:
mlMatchData.dtypes

matchid                    int64
matchdata                 object
team_0_teamRef            object
team_1_teamRef            object
winner                    object
matchdate         datetime64[ns]
dtype: object

In [177]:
def games_won_before(row,df,team):
    matchdate = row['matchdate']
    teamRef = row[team+'_teamRef']
    win = len(df[ ( df['matchdate']< matchdate ) & \
        ( df['winner']==team) & (df[team+'_teamRef']==teamRef) ])  
    #print(win)
    return win

In [178]:
mlMatchData['winner'].unique()

array(['team_1', 'team_0', 'draw'], dtype=object)

In [179]:
mlMatchData['team_0_won_before'] = mlMatchData.apply(lambda x: games_won_before(x,mlMatchData,'team_0'), axis=1)
mlMatchData['team_1_won_before'] = mlMatchData.apply(lambda x: games_won_before(x,mlMatchData,'team_1'), axis=1)

In [180]:
mlMatchData.head()

,matchid,matchdata,team_0_teamRef,team_1_teamRef,winner,matchdate,team_0_won_before,team_1_won_before
0,988751,20180728T160000+0100,t2081,t2089,team_1,2018-07-28 15:00:00,0,0
1,988750,20180728T160000+0100,t199,t4950,team_0,2018-07-28 15:00:00,0,0
2,988753,20180729T160000+0100,t2002,t212,team_1,2018-07-29 15:00:00,0,0
3,988752,20180729T160000+0100,t857,t280,team_0,2018-07-29 15:00:00,0,0
4,988754,20180729T160000+0100,t4948,t5666,team_0,2018-07-29 15:00:00,0,0


In [181]:
test = mlMatchData.mean(axis=0)

In [186]:
fields = list(allMatchData.columns[2:-1])
finalfields = [ x.replace("team_0","") for x in fields if "team_1" not in x ]
finalfields =finalfields[:-1]
len(finalfields)

229

In [208]:
#how many passes has this team made in the game before this game ( single)
def getLastWinMatchStat(row,df,curfield,team):
    teamRef = row[team+'_teamRef']
    matchdate = row['matchdate']
    lastWonMatch=df[ ( df['matchdate']< matchdate ) & \
        ( df['winner']==team) & (df[team+'_teamRef']==teamRef) ].sort_values(by=['matchdate'],ascending=False)
    if len(lastWonMatch)>0:
        return lastWonMatch.mean()[team+curfield]
    else:
        return 0

In [212]:
for curfield in finalfields:
    mlMatchData['team_0_lastwinmatch_'+curfield] = mlMatchData.apply(lambda x: getLastWinMatchStat(x,allMatchData,curfield,'team_0'), axis=1)
    mlMatchData['team_1__lastwinmatch_'+curfield] = mlMatchData.apply(lambda x: getLastWinMatchStat(x,allMatchData,curfield,'team_1'), axis=1)

In [213]:
mlMatchData.to_csv("mlMatchData.csv", encoding="utf-8") 

In [214]:
len(mlMatchData)

83